## Task 1: Nearest Neighbor Search

### Step 1: Loading and Parsing Data

In [14]:
# Importing necessary libraries 
import time
import math
import heapq
from rtree import index
import numpy as np

# Function to load the parking dataset from a file
def load_dataset(file_path):
    dataset = []  # Initialize an empty list to hold the dataset
    with open("Task1_Datasets/parking_dataset.txt", 'r') as f:  # Open the dataset file in read mode
        for line in f:  # Iterate over each line in the file
            parts = line.strip().split()  # Split the line into parts based on whitespace
            id, x, y = int(parts[0]), float(parts[1]), float(parts[2])  # Convert parts to appropriate data types
            dataset.append((id, x, y))  # Append the tuple (id, x, y) to the dataset list
    return dataset  # Return the loaded dataset

# Function to load the query points from a file
def load_queries(file_path):
    queries = []  # Initialize an empty list to hold the query points
    with open("Task1_Datasets/query_points.txt", 'r') as f:  # Open the query points file in read mode
        for line in f:  # Iterate over each line in the file
            parts = line.strip().split()  # Split the line into parts based on whitespace
            id, x, y = int(parts[0]), float(parts[1]), float(parts[2])  # Convert parts to appropriate data types
            queries.append((id, x, y))  # Append the tuple (id, x, y) to the queries list
    return queries  # Return the loaded query points


### Step 2: Implementing the Nearest Neighbor Search Algorithms

#### 1. Sequential Scan Based Method

In [15]:
# Calculating Euclidean distance between two points.
def euclidean_distance(point1, point2):
    return math.sqrt((point1[1] - point2[1])**2 + (point1[2] - point2[2])**2)  # Compute and return the Euclidean distance

# Finding the nearest neighbor for each query using the sequential scan method.
def sequential_scan(dataset, queries):
    results = []  # Initialize an empty list to hold the nearest neighbors
    start_time = time.time()  # Record the start time of the process
    for q in queries:  # Iterate over each query point
        nearest = min(dataset, key=lambda p: euclidean_distance(p, q))  # Find the nearest neighbor in the dataset
        results.append(nearest)  # Append the nearest neighbor to the results list
    total_time = time.time() - start_time  # Calculate the total time taken for all queries
    average_time = total_time / len(queries)  # Calculate the average time taken per query
    return results, total_time, average_time  # Return the nearest neighbors, total time taken, and average time per query



#### 2. Best First (BF) Algorithm Using R-tree

In [16]:
# Constructing an R-tree from the dataset.
def construct_rtree(dataset):
    p = index.Property()  # Create an R-tree property object
    rtree = index.Index(properties=p)  # Initialize an R-tree index with the specified properties
    for idx, point in enumerate(dataset):  # Iterate over each point in the dataset with its index
        # Insert the point into the R-tree with its index, bounding box, and object
        rtree.insert(idx, (point[1], point[2], point[1], point[2]), obj=point)
    return rtree  # Return the constructed R-tree

# Finding the nearest neighbor for each query using the Best First search method with an R-tree.
def best_first_search(rtree, queries):
    results = []  # Initialize an empty list to hold the nearest neighbors
    start_time = time.time()  # Record the start time of the process
    for q in queries:  # Iterate over each query point
        # Find the nearest neighbor using the R-tree and convert the result to a list, then get the object
        nearest = list(rtree.nearest((q[1], q[2], q[1], q[2]), 1, objects=True))[0].object
        results.append(nearest)  # Append the nearest neighbor to the results list
    total_time = time.time() - start_time  # Calculate the total time taken for all queries
    average_time = total_time / len(queries)  # Calculate the average time taken per query
    return results, total_time, average_time  # Return the nearest neighbors, total time taken, and average time per query



#### 3. BF with Divide-and-Conquer

In [17]:
# Dividing the dataset into two subspaces based on the median x value
def divide_dataset(dataset):
    mid_x = np.median([point[1] for point in dataset])  # Calculate the median x value of the dataset points
    # Create a sublist of points in the left subspace (x <= median x)
    left_subspace = [point for point in dataset if point[1] <= mid_x]
    # Create a sublist of points in the right subspace (x > median x)
    right_subspace = [point for point in dataset if point[1] > mid_x]
    return left_subspace, right_subspace  # Return two sublists of the dataset points

# Finding the nearest neighbor for each query using the BF with Divide-and-Conquer method.
def bf_with_divide_and_conquer(dataset, queries):
    left_subspace, right_subspace = divide_dataset(dataset)  # Divide the dataset into two subspaces
    rtree_left = construct_rtree(left_subspace)  # Construct an R-tree for the left subspace
    rtree_right = construct_rtree(right_subspace)  # Construct an R-tree for the right subspace
    
    results = []  # Initialize an empty list to hold the nearest neighbors
    start_time = time.time()  # Record the start time of the process
    for q in queries:  # Iterate over each query point
        # Find the nearest neighbor in the left subspace using the R-tree
        nearest_left = list(rtree_left.nearest((q[1], q[2], q[1], q[2]), 1, objects=True))[0].object
        # Find the nearest neighbor in the right subspace using the R-tree
        nearest_right = list(rtree_right.nearest((q[1], q[2], q[1], q[2]), 1, objects=True))[0].object
        # Compare distances to determine the closer neighbor
        if euclidean_distance(nearest_left, q) < euclidean_distance(nearest_right, q):
            results.append(nearest_left)  # Append the nearest neighbor from the left subspace
        else:
            results.append(nearest_right)  # Append the nearest neighbor from the right subspace
    total_time = time.time() - start_time  # Calculate the total time taken for all queries
    average_time = total_time / len(queries)  # Calculate the average time taken per query
    return results, total_time, average_time  # Return the nearest neighbors, total time taken, and average time per query



In [18]:
# Saving the results into the file.
# Arguments for save_results:
"""
results : List of nearest neighbors for each query.
method_name : Name of the method used to obtain the results.
total_time : Total time taken for all queries (float).
average_time : Average time taken per query (float).
file_path : Path to the output file.
"""

def save_results(results, method_name, total_time, average_time, file_path):
    # Open the file in append mode and write the results
    with open(file_path, 'a') as f:
        f.write(f"Results for {method_name}:\n")  # Write method name to the file
        # Write each result to the file
        for i, result in enumerate(results):
            f.write(f"id={result[0]}, x={result[1]}, y={result[2]} for query {i+1}\n")
        # Write total running time for all queries to the file
        f.write(f"Total running time for processing all queries: {total_time:.6f} seconds\n")
        # Write average time per query to the file
        f.write(f"Average time per query: {average_time:.6f} seconds\n\n")

# Define the file path
file_path = "Task_1_results.txt"
# Load data
parking_dataset = load_dataset('Task1_Datasets/parking_dataset.txt')
query_points = load_queries('Task1_Datasets/query_points.txt')

# Saving Sequential Scan result
seq_results, seq_total_time, seq_avg_time = sequential_scan(parking_dataset, query_points)
# Call save_results function to save the Sequential Scan result
save_results(seq_results, 'Sequential Scan Based', seq_total_time, seq_avg_time, file_path)

# Saving Best First Search using R-tree result
rtree = construct_rtree(parking_dataset)
bf_results, bf_total_time, bf_avg_time = best_first_search(rtree, query_points)
# Call save_results function to save the Best First Search using R-tree result
save_results(bf_results, 'BF Algorithm', bf_total_time, bf_avg_time, file_path)

# Saving BF with Divide-and-Conquer result
bf_dc_results, bf_dc_total_time, bf_dc_avg_time = bf_with_divide_and_conquer(parking_dataset, query_points)
# Call save_results function to save the BF with Divide-and-Conquer result
save_results(bf_dc_results, 'BF with Divide-and-Conquer', bf_dc_total_time, bf_dc_avg_time, file_path)
